## Todo List
- [x] Checar se formato de datas conferem.
- [x] Remover UGs sem liberação OC
- [x] Considerar coluna montagem. Onde estiver vazio, pegar valor de Torres para EOL, no skate.
- [ ] Separar um cojunto aleatório para teste
- [ ] Calcular tempos de construção por marco: (IO_realizado - OC_realizado), (CC_realizado - OC_realizado) ...etc  (ordem:IO,CC,ME,OT)
- [ ] Calcular atraso da operação comercial (OC_realizado - OC_outorgado)
- [ ] Calcular média e dp do tempo de construção.
- [ ] Calcular data dos marcos medios (OC_outorgado - tempo_medio_de_construcao) por fonte.
- [ ] Calcular atrasos com base nos marcos medios
- [ ] Calcular media e dp dos atrasos. 
- [ ] Normalizar os atrasos calculados (atraso - atraso_medio)/dp
- [ ] Construir matriz de correlação de Pearson
- [ ] Calcular os coeficientes para regressão linear
- [ ] Para UHE pegar todas UGs, para as demais pegar a primeira UG apenas 
- [ ] Comparar pegando primeira UG e media por grupo



## Importa bibliotecas

In [1]:
import pandas as pd
import numpy as np
import re
import pyodbc
from pprint import pprint
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy import stats

##  Cria conexões com os bancos de dado

In [2]:
import pyodbc
bdworgs = pyodbc.connect('Driver={SQL Server};'
                      'Server=bduorgs,5468;'
                      'Database=SFG_BD;'
                      'Trusted_Connection=yes;')

skate = pyodbc.connect('Driver={SQL Server};'
                      'Server=SAG003\SBD172;'
                      'Database=FiscalizacaoGeracao;'
                      'Trusted_Connection=yes;')

## Baixando banco de dados

In [3]:
skate_ug_command = """
SELECT [IdeUsinaOutorga]
      ,[CodCegFormatado]
      ,[SigTipoGeracao]
      ,[NomUsina]
      ,[NumUgUsina]
      ,[MdaPotenciaUnitaria]
      ,[MdaPotenciaLiberadaTeste]
      ,[MdaPotenciaLiberadaComercial]
      ,[NumDespachoTeste]
      ,[numDespachoComercial]
      ,[IdcMonitorada]
      ,[DatDescidaoutorgadaRotor]
      ,[DatRealizacaoSFGRotor]
      ,[DatInicioOpTesteOutorgada]
      ,[DatLiberacaoSFGTeste]
      ,[DatUGInicioOpComerOutorgado]
      ,[DatLiberOpComerRealizado]
      ,[DatPrevisaoSFGComercial]
  FROM [FiscalizacaoGeracao].[SKT].[vMonitoramentoUG]
"""
skate_ug = pd.read_sql_query(skate_ug_command,skate)
skate_ug.to_feather("skate_ug.feather")


id_tipo_usina = pd.read_sql_query("""
SELECT [IdeAgente]
      ,[SigTipoGeracao]
  FROM [FiscalizacaoGeracao].[SKT].[vUsinaOutorgas]
""",skate)
id_tipo_usina.to_feather("id_tipo_usina.feather")


skate_usinas_command = """
SELECT [IdeUsinaOutorga]
      ,[DatOutorgaUsina]
      ,[UFUsina]
      ,[IdcDescOrigemCombustivel]
      ,[SigTipoGeracao]
      ,[CodCegFormatado]
      ,[NomUsina]
      ,[VlrPotenciaOutorgada]
      ,[VlrPotenciaFiscalizada]
      ,[DscFaseUsina]
      ,[VlrGarantiaFisica]
      ,[DscTipoOutorga]
      ,[DscAtoOutorga]
      ,[NomOrgaoOutorgante]
      ,[NumNumeroAto]
      ,[DatEmissaoAto]
      ,[DscProprietario]
      ,[DscTipoConexao]
      ,[NomConexao]
      ,[MdaTensaoConexao]
      ,[NomEmpresaConexao]
      ,[NumCNPJ]
      ,[IdcUsinaMonitorada]
      ,[IdcViabilidade]
      ,[IdcSituacaoObra]
      ,[IdcObraParalisada]
      ,[IdcSemPrevisão]
      ,[DatPrevisaoIniciobra]
      ,[DatContratoEPCOutorgado]
      ,[DatRecursoFinanceiroOutorgado]
      ,[DatCanteiroObraOutorgado]
      ,[DatCanteiroObraRealizado]
      ,[DatInicioObraOutorgado]
      ,[DatInicioObraRealizado]
      ,[DatConcretagemOutorgado]
      ,[DatConcretagemRealizado]
      ,[DatDesvioRioOutorgado]
      ,[DatDesvioRioRealizado]
      ,[DatMontagemOutorgado]
      ,[DatMontagemRealizado]
      ,[DatConclusaoTorresOutorgado]
      ,[DatConclusaoTorresRealizado]
      ,[DatEnchimentoOutorgado]
      ,[DatEnchimentoRealizado]
      ,[DatComissionamentoUGRealizado]
      ,[DatSisTransmissaoOutorgado]
      ,[DatSisTransmissaoRealizado]
      ,[DatConclusaoSisTransOutorgado]
      ,[DatConclusaoSisTransRealizado]
      ,[DscJustificativaPrevisao]
      ,[DatMonitoramento]
      ,[DscObservacaoFiscal]
      ,[NumProcessoSicnet]
      ,[DscAssuntoSicnet]
      ,[DscComercializacaoEnergia]
      ,[DscSistema]
      ,[DatConclusaoTransporteRealizado]
      ,[IdcSituacaoCronograma]
  FROM [FiscalizacaoGeracao].[SKT].[vMonitoramentoUsina]
"""
skate_usinas = pd.read_sql_query(skate_usinas_command,skate)
skate_usinas.to_feather("skate_usinas.feather")

### Corrigindo banco de dados bdworgs


In [4]:
bdworgs = pd.read_excel("./bdworgs.xlsx") # Banco de dados com correções é importado
bdworgs.Num_UG = bdworgs.Num_UG.astype(str) 
bdworgs.to_feather("./bdworgs_corrigido.feather") 

In [5]:
def corrige_ug(ug): # Função criada para corrigir nomes das UGs
    ug = ug.replace(" ","")
    ug = ug.replace("e",",")
    ug = ug.replace("-","a")
    ug = re.sub(r",+",r",",ug)
    ug = re.sub(r"([,ae])(0)",r"\1",ug)
    ug = re.sub(r"^0",r"",ug)
    ug = ug.split("a")
    lista = []
    for i in range(len(ug)-1):
        menor = int(ug[i].split(",")[-1])
        maior = int(ug[i+1].split(",")[0])
        list_nums = list(range(menor+1,maior))
        lista += list_nums
    ug = [el.split(",") for el in ug]
    ug = [int(el2) for el1 in ug for el2 in el1]
    ug+= lista
    return (sorted(ug))

### Carrega bancos de dados

In [6]:
# Carrega bdworgs
bdworgs = pd.read_feather("./bdworgs_corrigido.feather")

# Carrega dois bancos de dados do skate
skate_ug = pd.read_feather("./skate_ug.feather")
skate_usinas = pd.read_feather("./skate_usinas.feather")

In [7]:
mask = bdworgs.Montagem_Outorga.isnull() 
bdworgs.loc[mask,"Montagem_Outorga"] = bdworgs.loc[mask,"Torres_Outorga"]

mask = bdworgs.Montagem_Outorga.isnull() 
bdworgs.loc[mask,"Montagem_Real"] = bdworgs.loc[mask,"Torres_Real"]



In [8]:
bdworgs

,Idempreendimento,Id_UG,Num_UG,Pot_UG,OpTest_Outor_UG,OpCom_Outor_UG,OpTest_R_UG,OpCom_R_UG,UF,PotenciaOutorgada,...,Montagem_Real,Enchimento_Real,Torres_Real,observacao1,observacao2,ClasseCombustivel,DscComercializacaoEnergia,DscSistema,Skate,FonteDataOC
0,30352,979,1 a 14,29.40,2013-06-10,2013-07-14,2014-02-04,2014-03-29,CE,29.40,...,None,None,2013-09-24,Relatório de enviado por e-mail em 3/2/14.,None,Eólica,ACR,Sistema Interligado Nacional,não,plan
1,30353,980,1 e 2,2.89,2014-04-04,2014-05-08,2012-05-17,2012-07-31,SC,2.89,...,2012-01-10,2012-04-25,None,None,None,Hídrica,Fora do ACR,Sistema Interligado Nacional,não,plan
2,30354,981,1,611.11,2016-01-31,2016-03-31,2016-03-23,2016-04-20,PA,11233.10,...,2013-11-12,2015-11-24,None,Dsp. 3.209/2019 OC UG 18\nDsp. 2.953/2019 OT U...,out/19 previsão da ug18 adiantada pela veloci...,Hídrica,ACR,Sistema Interligado Nacional,não,plan
3,30354,982,2,611.11,2016-03-31,2016-05-31,2016-06-28,2016-07-16,PA,11233.10,...,2013-11-12,2015-11-24,None,Dsp. 3.209/2019 OC UG 18\nDsp. 2.953/2019 OT U...,out/19 previsão da ug18 adiantada pela veloci...,Hídrica,ACR,Sistema Interligado Nacional,não,plan
4,30354,983,3,611.11,2016-05-31,2016-07-31,2016-09-20,2016-11-02,PA,11233.10,...,2013-11-12,2015-11-24,None,Dsp. 3.209/2019 OC UG 18\nDsp. 2.953/2019 OT U...,out/19 previsão da ug18 adiantada pela veloci...,Hídrica,ACR,Sistema Interligado Nacional,não,plan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2370,30350,974,1,16.00,None,None,2011-05-25,2011-06-01,RS,30.00,...,2011-04-15,None,2011-03-14,None,None,Eólica,ACR,Sistema Interligado Nacional,não,plan
2371,30350,975,2,12.00,None,None,2011-05-25,2011-07-01,RS,30.00,...,2011-04-15,None,2011-03-14,None,None,Eólica,ACR,Sistema Interligado Nacional,não,plan
2372,30350,976,3,2.00,None,None,2011-05-25,2011-08-01,RS,30.00,...,2011-04-15,None,2011-03-14,None,None,Eólica,ACR,Sistema Interligado Nacional,não,plan
2373,30351,977,1 e 4,4.00,2012-06-26,2012-06-30,2014-03-01,2014-03-29,RN,8.00,...,2012-06-28,None,2012-06-01,None,Alterada Previsão Aneel - Informações do SIGET...,Eólica,ACR,Sistema Interligado Nacional,não,plan


### Tabela com associações de nomes entre os bancos de dados

In [9]:
# Carrega tabela de associações
associassoes = pd.read_excel("associassoes.xlsx")
associassoes

,bdworgs,skate
0,Idempreendimento,IdeUsinaOutorga
1,OpTest_Outor_UG,DatInicioOpTesteOutorgada
2,OpCom_Outor_UG,DatUGInicioOpComerOutorgado
3,OpTest_R_UG,DatLiberacaoSFGTeste
4,OpCom_R_UG,DatLiberOpComerRealizado
5,NaN,DatOutorgaUsina
6,UF,UFUsina
7,NaN,SigTipoGeracao
8,PotenciaOutorgada,VlrPotenciaOutorgada
9,Obras_Outorga,DatInicioObraOutorgado


In [10]:
# Cria dicionário dos nomes das colunas para poder relacionar os nomes das colunas
# Dicionário de colunas bdworgs > skate
dict_bdworgs_skate = dict(zip(associassoes.dropna()["bdworgs"],associassoes.dropna()["skate"]))
# Dicionário de colunas skate > bdworgs
dict_skate_bdworgs = dict(zip(associassoes.dropna()["skate"],associassoes.dropna()["bdworgs"]))

In [11]:
# Colunas carregados do skate
skate_cols = [el for el in associassoes["skate"].dropna()]
# Colunas carregadas do bdworgs
bdworgs_cols =[el for el in associassoes["bdworgs"].dropna()]
skate_ug_cols = []
skate_usinas_col = []
for col in skate_cols:
    if col in skate_ug.columns:
        skate_ug_cols.append(col)
    if col in skate_usinas.columns:
        skate_usinas_col.append(col)
        
bdworgs = bdworgs[bdworgs_cols]
skate_ug = skate_ug[skate_ug_cols]
skate_usinas = skate_usinas[skate_usinas_col]
bdworgs = bdworgs.rename(columns=dict_bdworgs_skate)
bdworgs.sample(5)

,IdeUsinaOutorga,DatInicioOpTesteOutorgada,DatUGInicioOpComerOutorgado,DatLiberacaoSFGTeste,DatLiberOpComerRealizado,UFUsina,VlrPotenciaOutorgada,DatInicioObraOutorgado,DatInicioObraRealizado,DatConcretagemOutorgado,...,DatMontagemOutorgado,DatMontagemRealizado,Torres_Outorga,Torres_Real,DatEnchimentoOutorgado,DatEnchimentoRealizado,DatSisTransmissaoOutorgado,DatSisTransmissaoRealizado,Num_UG,Pot_UG
1102,33647,2018-07-07,2019-01-01,2018-12-11,2019-02-14,RN,21.0,None,2017-01-15,2016-02-12,...,2017-04-21,None,2017-04-21,2018-08-15,None,None,2017-02-24,2018-09-15,1 a 10,21.0
717,31663,2018-04-26,2018-04-30,2016-12-01,2017-02-02,RS,15.0,2017-01-28,2016-01-15,2017-03-19,...,2017-08-26,None,2017-08-26,2016-11-15,None,None,2017-03-29,2016-02-11,1 a 5,15.0
371,30914,2015-11-30,2015-12-31,2015-11-06,2015-12-05,RN,24.0,2015-05-01,2014-12-01,2015-07-01,...,2015-10-01,None,2015-10-01,2015-04-21,None,None,2015-03-01,2015-02-15,1 e 3,6.0
1163,33784,2017-07-01,2017-08-01,2018-08-30,2018-11-08,BA,27.0,2016-10-01,2017-11-30,None,...,2017-01-02,2018-01-15,None,None,None,None,2016-12-01,2018-04-13,1 a 27,27.0
418,31034,None,None,2014-05-20,2014-07-11,PR,42.0,None,2013-05-07,None,...,None,None,None,None,None,None,None,None,1,42.0


### Adiciona as seguintes  colunas no bdworgs:
- Quant_UG  &rarr; Quantidade de UGs agrupadas
- SigTipoGeracao &rarr; Tipo de geração


In [12]:
# Insere coluna Quant_UG em bdworgs
bdworgs.insert(3,"Quant_UG",bdworgs["Num_UG"].apply(lambda x : len(corrige_ug(x))))
# Remove coluna Num_UG
bdworgs = bdworgs.drop("Num_UG",axis=1)

ceg_df = pd.read_excel("./Relatorio_Outorgas.xlsx")[["Identificador da Usina","Tipo de Geração"]]
dict_id_ceg = dict(zip(ceg_df["Identificador da Usina"],ceg_df["Tipo de Geração"]))
bdworgs["SigTipoGeracao"] = bdworgs["IdeUsinaOutorga"].map(dict_id_ceg)
bdworgs.sample(5)

,IdeUsinaOutorga,DatInicioOpTesteOutorgada,DatUGInicioOpComerOutorgado,Quant_UG,DatLiberacaoSFGTeste,DatLiberOpComerRealizado,UFUsina,VlrPotenciaOutorgada,DatInicioObraOutorgado,DatInicioObraRealizado,...,DatMontagemOutorgado,DatMontagemRealizado,Torres_Outorga,Torres_Real,DatEnchimentoOutorgado,DatEnchimentoRealizado,DatSisTransmissaoOutorgado,DatSisTransmissaoRealizado,Pot_UG,SigTipoGeracao
1184,33931,2018-10-01,2018-11-01,8,2018-12-08,2018-12-19,BA,30.0,2018-02-01,2018-04-03,...,2018-04-01,2018-06-15,None,None,None,None,2018-03-01,2018-05-15,8.00,UFV
1987,29597,2011-04-30,2011-06-30,1,2011-07-16,2011-09-14,MT,261.0,None,2007-09-15,...,None,None,None,None,None,None,None,None,58.00,UHE
859,32013,2016-06-01,2016-06-06,1,2016-09-22,2016-10-26,CE,18.9,None,2015-07-15,...,None,2015-11-18,None,2015-11-18,None,None,None,2016-03-10,2.70,EOL
361,30898,2015-12-01,2015-12-31,3,2014-11-13,2015-03-24,RN,30.0,2015-05-01,2014-01-15,...,2015-10-01,2014-10-02,2015-10-01,2014-08-15,None,None,2015-03-01,2013-08-19,9.00,EOL
232,30739,2014-06-13,2014-07-12,7,2016-02-06,2016-02-16,BA,22.2,2013-08-13,2014-08-15,...,2014-03-06,None,2014-03-06,2015-06-02,None,None,2013-09-08,2015-03-15,12.95,EOL


### No dataframes skate_usina e bdworgs são feitas as seguintes alterações:
 - Onde não não existe data na coluna DatMontagemOutorgado e existe na coluna DatConclusaoTorresOutorgado, o valor que está na coluna DatConclusaoTorresOutorgado passa para a coluna DatMontagemOutorgado.
 - Mesmo processo feito para as colunas DatConclusaoTorresOutorgado/DatMontagemOutorgado é feito para as colunas DatConclusaoTorresRealizado/DatMontagemRealizado

### No BD skate_ug, linhas com todas as colunas iguais tais como IdeUsinaOutorga, DatInicioOpTesteOutorgada... (exceto  MdaPotenciaUnitaria, que não necessariamente tem que ser igual em todas as linhas)  são agrupadas e são criadas duas outras colunas: 
- Quant_UG -> &rarr; Quantidade de UGs agrupadas
- Pot_UG -> &rarr; Soma da potência das UGs que foram agrupadas

#### Entrada: skate_ug
#### Saída: skate_ug_grouped

In [13]:
skate_ug_group_list = [x for x  in skate_ug.columns.to_list() if x not in ["MdaPotenciaUnitaria"]]
skate_ug_grouped = skate_ug.groupby(skate_ug_group_list,dropna=False).agg(
    Quant_UG = ("MdaPotenciaUnitaria","count"),
    Pot_UG = ("MdaPotenciaUnitaria","sum")
    
).reset_index()

### Realiza o processo acima para o bdworgs:
- Quant_UG -> &rarr; Quantidade de UGs agrupadas. Nesse caso, como o bdworgs, já possuía a coluna Quant_UG, os valores dessa coluna foram somados para os valores agrupados.
- Pot_UG -> &rarr; Soma da potência dos conjuntos de UGs que foram agrupados.

#### Entrada: bdworgs
#### Saída: bdworgs_ug_grouped

In [14]:
dummy_bdworgs = bdworgs.rename(columns={"Quant_UG":"Quant_UG_","Pot_UG":"Pot_UG_"})
dummy_bdworgs_group_list = [x for x  in dummy_bdworgs.columns.to_list() if x not in ["Quant_UG_","Pot_UG_"]]

bdworgs_ug_grouped = dummy_bdworgs.groupby(dummy_bdworgs_group_list,dropna=False).agg(
    Quant_UG = ("Quant_UG_","sum"),
    Pot_UG = ("Pot_UG_","sum")
    
).reset_index() 

### Checa se a quantidade de potência e de usinas nos dataframes iniciais e agrupados são iguais.

In [15]:
print(skate_ug_grouped.Quant_UG.sum() - skate_ug.MdaPotenciaUnitaria.count())
print(skate_ug_grouped.Pot_UG.sum() - skate_ug.MdaPotenciaUnitaria.sum())
print(bdworgs_ug_grouped.Pot_UG.sum() - bdworgs_ug_grouped.Pot_UG.sum())
print(bdworgs_ug_grouped.Quant_UG.count() - bdworgs_ug_grouped.Quant_UG.count())


0
0.0
0.0
0


### Junta bancos de dados skate_ug e skate_usina pela coluna IdeUsinaOutorga. Todos os atributos de cada usina são passados para cada UG pertecente a usina.

#### Entrada: skate_ug_grouped & skate_usinas
#### Saída: skate_merged

In [16]:
skate_merged  = pd.merge(skate_ug_grouped.drop(["SigTipoGeracao"],axis=1),
        skate_usinas,
        on="IdeUsinaOutorga",
         how="outer")
skate_merged.sample(10)

,IdeUsinaOutorga,DatInicioOpTesteOutorgada,DatUGInicioOpComerOutorgado,DatLiberacaoSFGTeste,DatLiberOpComerRealizado,Quant_UG,Pot_UG,DatOutorgaUsina,UFUsina,SigTipoGeracao,...,DatInicioObraOutorgado,DatInicioObraRealizado,DatConcretagemOutorgado,DatConcretagemRealizado,DatMontagemOutorgado,DatMontagemRealizado,DatEnchimentoOutorgado,DatEnchimentoRealizado,DatSisTransmissaoOutorgado,DatSisTransmissaoRealizado
2712,47063,2023-01-01,2023-03-01,NaN,NaN,16,27020.00,2020-09-08,PB,UFV,...,2022-06-26,None,None,None,2022-08-06,None,None,None,2022-11-08,None
1991,38322,2023-12-02,2024-01-01,2021-10-02,2021-10-27,1,4200.00,2019-01-09,RN,EOL,...,2023-01-31,2020-09-04,2023-04-01,2020-10-15,2023-08-04,2021-06-01,None,None,2023-01-31,2020-06-23
1972,38320,2023-12-17,2024-01-01,2021-09-11,2021-09-29,2,8400.00,2019-01-09,RN,EOL,...,2023-01-31,2020-08-13,2023-04-01,2020-09-28,2023-08-04,2021-05-18,None,None,2023-01-31,2020-06-23
447,32482,2023-02-12,2023-02-27,NaN,NaN,1,5500.00,2022-02-01,CE,EOL,...,2022-05-01,None,2022-06-22,None,2022-12-13,None,None,None,2022-04-01,None
2646,46737,2023-12-01,2024-01-31,NaN,NaN,12,30000.00,2021-09-21,BA,UFV,...,2022-08-01,None,None,None,2023-03-01,None,None,None,2023-02-01,None
707,33532,2021-11-17,2022-01-01,2020-08-27,2020-09-17,3,12600.00,2018-08-15,BA,EOL,...,2021-01-31,2019-06-01,2021-04-01,2019-11-15,2021-08-04,2020-06-15,None,None,2021-01-31,2019-07-15
1704,37720,2018-11-10,2018-11-19,2019-10-19,2020-02-01,17,8836.00,2017-07-25,AM,UTE,...,2018-04-26,2018-11-15,None,None,2018-06-24,2019-05-03,None,None,2018-07-24,2019-07-01
2613,46561,2024-12-15,2025-01-15,NaN,NaN,12,41244.00,2021-08-24,MG,UFV,...,2024-03-15,None,None,None,2024-07-15,None,None,None,2024-07-15,None
2692,46927,2023-12-01,2024-01-31,NaN,NaN,24,40536.00,2022-02-22,RN,UFV,...,2022-09-06,None,None,None,2023-01-04,None,None,None,2022-08-03,None
840,34105,2021-01-16,2021-02-16,2021-03-30,2021-10-15,2,6666.66,2018-05-09,PE,UFV,...,2020-05-01,2019-11-01,None,None,2020-05-01,2020-03-15,None,None,2020-07-30,2020-06-15


### Converte colunas cujos nomes iniciam em "Dat" para tipo de data. É necessário que as colunas estejam nomeadas no padrão do skate.

In [17]:
colunas_datas = []
for col in bdworgs_ug_grouped.columns:
    if col[0:3] == "Dat":
        colunas_datas.append(col)
        print(col)
        skate_merged[col] = pd.to_datetime(skate_merged[col],format="%Y-%m-%d",errors="coerce")
        bdworgs_ug_grouped[col] = pd.to_datetime(bdworgs_ug_grouped[col],format="%Y-%m-%d",errors="coerce")

DatInicioOpTesteOutorgada
DatUGInicioOpComerOutorgado
DatLiberacaoSFGTeste
DatLiberOpComerRealizado
DatInicioObraOutorgado
DatInicioObraRealizado
DatConcretagemOutorgado
DatConcretagemRealizado
DatMontagemOutorgado
DatMontagemRealizado
DatEnchimentoOutorgado
DatEnchimentoRealizado
DatSisTransmissaoOutorgado
DatSisTransmissaoRealizado


### Checa quais colunas não estão presentes em cada um dos dataframes

In [18]:
print("Colunas que estão no skate, porém não no bdworgs:")
for col in skate_merged.columns:
    if col not in bdworgs.columns:
        print(col)
        
print("-"*10)
print("Colunas que estão no bdworgs, porém não no skate:")
for col in bdworgs.columns:
    if col not in skate_merged.columns:
        print(col)

Colunas que estão no skate, porém não no bdworgs:
DatOutorgaUsina
----------
Colunas que estão no bdworgs, porém não no skate:
Torres_Outorga
Torres_Real


### Junta BDs skate e bdworgs.
#### Entrada: skate_merged & bdworgs_ug_grouped
#### Saída: skate_merged

In [19]:
bdworgs_ug_grouped["origem"] = "bdworgs"
skate_merged["origem"] = "skate"
bd_combined = pd.concat([bdworgs_ug_grouped,skate_merged],axis=0,ignore_index=True)
#bd_combined = bdworgs_ug_grouped.copy(deep=True)
bd_combined["origem"] = bd_combined["origem"].astype("category")
bd_combined.sample(10)

,IdeUsinaOutorga,DatInicioOpTesteOutorgada,DatUGInicioOpComerOutorgado,DatLiberacaoSFGTeste,DatLiberOpComerRealizado,UFUsina,VlrPotenciaOutorgada,DatInicioObraOutorgado,DatInicioObraRealizado,DatConcretagemOutorgado,...,Torres_Real,DatEnchimentoOutorgado,DatEnchimentoRealizado,DatSisTransmissaoOutorgado,DatSisTransmissaoRealizado,SigTipoGeracao,Quant_UG,Pot_UG,origem,DatOutorgaUsina
556,29480,2007-05-01,2008-06-30,2009-01-30,2009-01-31,MG,63.155,2006-10-01,2006-12-15,NaT,...,NaN,NaT,NaT,NaT,NaT,UTE,1,63.155,bdworgs,NaN
1481,31355,2015-08-01,2015-09-01,2017-09-19,2017-12-21,BA,28.000,2014-07-01,2015-01-28,2014-11-01,...,2015-07-17,NaT,NaT,2014-11-01,2015-03-15,EOL,14,28.000,bdworgs,NaN
5007,46884,2023-04-20,2023-05-30,NaT,NaT,BA,48000.000,2022-08-01,NaT,NaT,...,NaN,NaT,NaT,2022-12-15,NaT,UFV,24,48000.000,skate,2021-10-05
4490,40663,2024-07-01,2024-08-01,NaT,NaT,MG,3000.000,2023-05-01,2021-09-14,2023-06-01,...,NaN,2024-04-01,NaT,2023-09-01,NaT,CGH,1,1000.000,skate,2020-05-25
3259,34511,2021-08-26,2021-10-08,2021-10-14,2021-11-13,BA,46200.000,2020-07-15,2020-11-30,2020-07-15,...,NaN,NaT,NaT,2020-09-08,2021-01-15,EOL,2,8400.000,skate,2019-09-17
1760,31810,2016-11-20,2017-01-01,2016-11-26,2016-12-28,RN,30.000,2016-03-01,2016-02-15,2016-06-01,...,2016-07-15,NaT,NaT,2016-02-01,2016-03-09,EOL,15,30.000,bdworgs,NaN
1658,31646,2018-04-26,2018-04-30,2017-04-12,2017-05-13,RS,15.000,2017-01-28,2015-09-15,2017-03-19,...,2017-03-07,NaT,NaT,2017-03-29,2016-02-11,EOL,5,15.000,bdworgs,NaN
4262,38310,2023-12-16,2024-01-01,2021-10-08,2021-11-17,RN,63000.000,2023-01-31,2021-01-15,2023-04-01,...,NaN,NaT,NaT,2023-01-31,2020-06-23,EOL,2,8400.000,skate,2019-01-08
3821,37102,2023-12-17,2024-01-01,2020-04-03,2020-05-14,BA,50400.000,2023-01-31,2019-07-15,2023-04-01,...,NaN,NaT,NaT,2023-01-31,2019-05-01,EOL,1,4200.000,skate,2018-12-27
689,29719,NaT,NaT,NaT,2013-02-06,PE,381.255,2011-06-01,2010-05-15,NaT,...,NaN,NaT,NaT,NaT,NaT,UTE,1,22.427,bdworgs,NaN


In [20]:
bd_combined.to_parquet("bd_combined.gzip")

In [21]:
bdworgs[bdworgs.IdeUsinaOutorga.duplicated(keep=False)].head(25)

,IdeUsinaOutorga,DatInicioOpTesteOutorgada,DatUGInicioOpComerOutorgado,Quant_UG,DatLiberacaoSFGTeste,DatLiberOpComerRealizado,UFUsina,VlrPotenciaOutorgada,DatInicioObraOutorgado,DatInicioObraRealizado,...,DatMontagemOutorgado,DatMontagemRealizado,Torres_Outorga,Torres_Real,DatEnchimentoOutorgado,DatEnchimentoRealizado,DatSisTransmissaoOutorgado,DatSisTransmissaoRealizado,Pot_UG,SigTipoGeracao
2,30354,2016-01-31,2016-03-31,1,2016-03-23,2016-04-20,PA,11233.1,2011-05-31,2011-08-18,...,2013-10-31,2013-11-12,None,None,2014-12-15,2015-11-24,2014-03-31,2014-09-30,611.11,UHE
3,30354,2016-03-31,2016-05-31,1,2016-06-28,2016-07-16,PA,11233.1,2011-05-31,2011-08-18,...,2013-10-31,2013-11-12,None,None,2014-12-15,2015-11-24,2014-03-31,2014-09-30,611.11,UHE
4,30354,2016-05-31,2016-07-31,1,2016-09-20,2016-11-02,PA,11233.1,2011-05-31,2011-08-18,...,2013-10-31,2013-11-12,None,None,2014-12-15,2015-11-24,2014-03-31,2014-09-30,611.11,UHE
5,30354,2016-07-31,2016-09-30,1,2016-12-09,2017-01-18,PA,11233.1,2011-05-31,2011-08-18,...,2013-10-31,2013-11-12,None,None,2014-12-15,2015-11-24,2014-03-31,2014-09-30,611.11,UHE
6,30354,2016-09-30,2016-11-30,1,2017-03-10,2017-04-20,PA,11233.1,2011-05-31,2011-08-18,...,2013-10-31,2013-11-12,None,None,2014-12-15,2015-11-24,2014-03-31,2014-09-30,611.11,UHE
7,30354,2016-11-30,2017-01-31,1,2017-06-28,2017-07-19,PA,11233.1,2011-05-31,2011-08-18,...,2013-10-31,2013-11-12,None,None,2014-12-15,2015-11-24,2014-03-31,2014-09-30,611.11,UHE
8,30354,2017-01-31,2017-03-31,1,2017-10-10,2017-10-28,PA,11233.1,2011-05-31,2011-08-18,...,2013-10-31,2013-11-12,None,None,2014-12-15,2015-11-24,2014-03-31,2014-09-30,611.11,UHE
9,30354,2017-03-31,2017-05-31,1,2017-12-23,2018-02-01,PA,11233.1,2011-05-31,2011-08-18,...,2013-10-31,2013-11-12,None,None,2014-12-15,2015-11-24,2014-03-31,2014-09-30,611.11,UHE
10,30354,2017-05-31,2017-07-31,1,2018-05-25,2018-06-13,PA,11233.1,2011-05-31,2011-08-18,...,2013-10-31,2013-11-12,None,None,2014-12-15,2015-11-24,2014-03-31,2014-09-30,611.11,UHE
11,30354,2017-07-31,2017-09-30,1,2018-09-21,2018-10-05,PA,11233.1,2011-05-31,2011-08-18,...,2013-10-31,2013-11-12,None,None,2014-12-15,2015-11-24,2014-03-31,2014-09-30,611.11,UHE
